In [ ]:
pip install gegravity

In [ ]:
import gegravity as ge
import pandas as pd
# Increase number of columns printed for a pandas DataFrame
pd.set_option("display.max_columns", None)
pd.set_option('display.width', 1000)
import gme as gme

In [ ]:
from google.colab import files
uploaded = files.upload()

Saving gravity_data_final.csv to gravity_data_final (2).csv


In [ ]:
gravity_data_location = "gravity_data_final.csv"
grav_data = pd.read_csv(gravity_data_location)

In [ ]:
print(grav_data.head())

  ImporterCode ExporterCode  Year  GrossImports_1000USD          ImporterName  ExporterName  contig  comlang_off       dist    lndist   ExporterGDP  ImporterGDP  pta  International
0          ARE          SAU  2017          6.739881e+06  United Arab Emirates  Saudi Arabia       1            1   774.5853  6.652328  6.885861e+05  385605.5069    0              1
1          ARE          LAO  2017          2.255521e+04  United Arab Emirates       Lao PDR       0            0  5029.9320  8.523162  1.707116e+04  385605.5069    0              1
2          ARE          THA  2017          3.677137e+06  United Arab Emirates      Thailand       0            0  4967.7960  8.510732  4.563570e+05  385605.5069    0              1
3          ARE          BRN  2017          4.460190e+02  United Arab Emirates        Brunei       0            0  6813.5570  8.826670  1.212810e+04  385605.5069    0              1
4          ARE          DEU  2017          1.151235e+07  United Arab Emirates       Germany    

In [ ]:
gme_data = gme.EstimationData(grav_data, # Dataset
                              imp_var_name="ImporterName", # Importer column name
                              exp_var_name="ExporterName", # Exporter column name
                              year_var_name = "Year",  # Year column name
                              trade_var_name="GrossImports_1000USD")  # Trade column name

In [ ]:
gme_model = gme.EstimationModel(gme_data, # Specify data to use
                                lhs_var="GrossImports_1000USD",                               # dependent, "left hand side" variable
                                rhs_var=["pta", "contig","comlang_off", # independent variables
                                         "lndist", "International"],
                                fixed_effects=[["ExporterName"],["ImporterName"]])

In [ ]:
print(grav_data.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 256 entries, 0 to 255
Data columns (total 14 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   ImporterCode          256 non-null    object 
 1   ExporterCode          256 non-null    object 
 2   Year                  256 non-null    int64  
 3   GrossImports_1000USD  256 non-null    float64
 4   ImporterName          256 non-null    object 
 5   ExporterName          256 non-null    object 
 6   contig                256 non-null    int64  
 7   comlang_off           256 non-null    int64  
 8   dist                  256 non-null    float64
 9   lndist                256 non-null    float64
 10  ExporterGDP           256 non-null    float64
 11  ImporterGDP           256 non-null    float64
 12  pta                   256 non-null    int64  
 13  International         256 non-null    int64  
dtypes: float64(5), int64(5), object(4)
memory usage: 28.1+ KB
None


In [ ]:
print(grav_data.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 256 entries, 0 to 255
Data columns (total 14 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   ImporterCode          256 non-null    object 
 1   ExporterCode          256 non-null    object 
 2   Year                  256 non-null    int64  
 3   GrossImports_1000USD  256 non-null    float64
 4   ImporterName          256 non-null    object 
 5   ExporterName          256 non-null    object 
 6   contig                256 non-null    int64  
 7   comlang_off           256 non-null    int64  
 8   dist                  256 non-null    float64
 9   lndist                256 non-null    float64
 10  ExporterGDP           256 non-null    float64
 11  ImporterGDP           256 non-null    float64
 12  pta                   256 non-null    int64  
 13  International         256 non-null    int64  
dtypes: float64(5), int64(5), object(4)
memory usage: 28.1+ KB
None


In [ ]:
gme_model.estimate()

Estimation began at 06:04 AM  on Nov 10, 2023
Omitted Regressors: ['ImporterName_fe_Vietnam']
Estimation completed at 06:04 AM  on Nov 10, 2023


/usr/local/lib/python3.10/dist-packages/gme/estimate/_ppml_estimation_and_diagnostics.py:254: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  adjusted_data_frame.drop(col, 1, inplace=True)
/usr/local/lib/python3.10/dist-packages/gme/estimate/_ppml_estimation_and_diagnostics.py:306: FutureWarning: The series.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  diagnostics = overfit_column.append(exclusion_column)


{'all': <statsmodels.genmod.generalized_linear_model.GLMResultsWrapper at 0x7d523628c070>}

In [ ]:
print(gme_model.results_dict['all'].summary())

                  Generalized Linear Model Regression Results                   
Dep. Variable:     GrossImports_1000USD   No. Observations:                  256
Model:                              GLM   Df Residuals:                      220
Model Family:                   Poisson   Df Model:                           35
Link Function:                      Log   Scale:                          1.0000
Method:                            IRLS   Log-Likelihood:            -2.6853e+08
Date:                  Fri, 10 Nov 2023   Deviance:                   5.3706e+08
Time:                          06:04:10   Pearson chi2:                 5.71e+08
No. Iterations:                      20   Pseudo R-squ. (CS):              1.000
Covariance Type:                    HC1                                         
                                           coef    std err          z      P>|z|      [0.025      0.975]
-------------------------------------------------------------------------------------

In [ ]:
ge_model = ge.OneSectorGE(gme_model,                   # gme gravity model
                       year = "2017",               # Year to use for model
                       expend_var_name = "ImporterGDP",       # Expenditure column name
                       output_var_name = "ExporterGDP",       # Output column name
                       reference_importer = "India",  # Reference importer
                       sigma = 5)                   # Elasticity of substitution

In [ ]:
test_diagnostics = ge_model.test_baseline_mr_function()
# See what is returned:
print(test_diagnostics.keys())
# Check the values of the model parameters computed from the baseline data, which should be numeric with no missing
# values
input_params = test_diagnostics['mr_params']
# Check one set of parameters, for example:
print(input_params['cost_exp_shr'])

dict_keys(['initial values', 'mr_params', 'function_value'])
[[1.74437651e-04 3.33122326e-03 3.75974143e-01 8.54565295e-02
  8.02002838e-02 2.13552390e-03 5.23222094e-02 7.01696848e-03
  4.99815985e-02 2.99710165e-02 5.08509491e-02 5.84182893e-02
  9.71786205e-03 3.68331491e-01 2.00523340e-02 1.39922576e-01]
 [1.01397761e-03 3.12574142e-04 3.96641556e-01 8.94791553e-02
  7.32114771e-02 3.10283345e-03 5.20427568e-02 9.28639039e-03
  4.43149757e-02 3.20563528e-02 5.30777457e-02 9.38142065e-02
  1.04840732e-02 3.73615471e-01 2.61533499e-02 1.57087399e-01]
 [3.72900160e-04 1.29243342e-03 4.74440210e-02 9.64520634e-02
  2.79533218e-02 1.05129023e-03 1.86465405e-02 3.52571552e-03
  2.01556282e-02 3.14795372e-02 1.97562968e-02 2.56759869e-02
  1.01867603e-02 4.09156063e-01 1.06278172e-02 8.43898747e-02]
 [2.63583763e-04 9.06714696e-04 2.99951014e-01 3.54642035e-02
  2.16082334e-02 6.88549790e-04 1.25677883e-02 2.47199040e-03
  1.29691982e-02 3.70339016e-02 1.33050556e-02 1.82743759e-02
  1.11

In [ ]:
rescale_eval = ge_model.check_omr_rescale(omr_rescale_range=3)
print(rescale_eval)


Trying OMR rescale factor of 0.001
Solving for baseline MRs...
The solution converged.

Trying OMR rescale factor of 0.01
Solving for baseline MRs...


/usr/local/lib/python3.10/dist-packages/gegravity/OneSectorGE.py:558: UserWarning: The iteration is not making good progress, as measured by the 
  improvement from the last five Jacobian evaluations.
  warn(solved_mrs.message)
/usr/local/lib/python3.10/dist-packages/gegravity/OneSectorGE.py:574: RuntimeWarning: invalid value encountered in double_scalars
  self.country_set[country].baseline_omr = 1 / (self.country_set[country]._baseline_omr_ratio ** sigma_inverse)
/usr/local/lib/python3.10/dist-packages/gegravity/OneSectorGE.py:573: RuntimeWarning: invalid value encountered in double_scalars
  self.country_set[country].baseline_imr = 1 / (self.country_set[country]._baseline_imr_ratio ** sigma_inverse)
/usr/local/lib/python3.10/dist-packages/gegravity/OneSectorGE.py:1596: RuntimeWarning: invalid value encountered in double_scalars
  omr = omr =(1/omr_ratio)**(1/(1-self.sigma))



Trying OMR rescale factor of 0.1
Solving for baseline MRs...
The solution converged.

Trying OMR rescale factor of 1
Solving for baseline MRs...
The solution converged.

Trying OMR rescale factor of 10
Solving for baseline MRs...
The solution converged.

Trying OMR rescale factor of 100
Solving for baseline MRs...
The solution converged.

Trying OMR rescale factor of 1000
Solving for baseline MRs...
The solution converged.
   omr_rescale omr_rescale (alt format)  solved                                            message  max_func_value  mean_func_value  reference_importer_omr
0        0.001                    10^-3    True                            The solution converged.    9.650586e-09     4.299126e-10                0.832179
1        0.010                    10^-2   False  The iteration is not making good progress, as ...    4.060634e-02    -2.278577e-03                     NaN
2        0.100                    10^-1    True                            The solution converged.    2.

In [ ]:
ge_model.build_baseline(omr_rescale=100)
# Examine the solutions for the baselin multilateral resistances
print(ge_model.baseline_mr.head(12))

Solving for baseline MRs...
The solution converged.
              baseline omr  baseline imr
country                                 
Brunei            0.834301      1.004164
Cambodia          0.824599      0.991985
China             0.911384      1.097498
Germany           0.851518      1.026177
India             0.832179      1.000000
Indonesia         0.853696      1.027662
Lao PDR           0.817530      0.982778
Malaysia          0.827806      0.996560
Myanmar           0.821161      0.987965
Philippines       0.820696      0.987795
Saudi Arabia      0.859434      1.035603
Singapore         0.816894      0.991761


In [ ]:
exp_data = ge_model.baseline_data.copy()

In [ ]:
# List of countries
countries = ["Brunei", "Cambodia", "Indonesia", "India", "Lao PDR", "Malaysia", "Myanmar", "Philppines", "Vietnam", "Singapore", "Thailand"]

# Import any necessary libraries here

# Define the GE model setup here

# Iterate through the list of countries
for country in countries:
    exp_data.loc[(exp_data["ImporterName"] == "India") & (exp_data["ExporterName"] == country), "pta"] = 0
    exp_data.loc[(exp_data["ImporterName"] == country) & (exp_data["ExporterName"] == "India"), "pta"] = 0

    # Define the experiment within the GE model
    ge_model.define_experiment(exp_data)

    # Examine the baseline and counterfactual trade costs
    print(ge_model.bilateral_costs.head(56))

                                   baseline trade cost  experiment trade cost  trade cost change (%)
ExporterName ImporterName                                                                           
Brunei       Brunei                           0.629399               0.629399               0.000000
             Cambodia                         3.658593               3.658593               0.000000
             China                            1.345483               1.345483               0.000000
             Germany                          0.951052               0.951052               0.000000
             India                            2.325215               1.251975             -46.156578
             Indonesia                        3.479491               3.479491               0.000000
             Lao PDR                          3.179007               3.179007               0.000000
             Malaysia                         4.063743               4.063743              

In [ ]:
ge_model.simulate()

Solving for conditional MRs...
The solution converged.
Solving full GE model...
The solution converged.


In [ ]:
country_results = ge_model.country_results

print(country_results)

                      factory gate price change (percent)  omr change (percent)  imr change (percent)  GDP change (percent)  welfare statistic  terms of trade change (percent)  output change (percent)  expenditure change (percent)  foreign exports change (percent)  foreign imports change (percent)  intranational trade change (percent)
country                                                                                                                                                                                                                                                                                                                                         
Brunei                                          -2.011565              2.052860             -0.616239             -1.403978           1.014240                        -1.403978                -2.011565                     -2.011565                         -2.036581                         -2.042620                            

In [ ]:
bilateral_results = ge_model.bilateral_trade_results

print(bilateral_results)

                                   baseline modeled trade  experiment trade  trade change (percent)
ExporterName ImporterName                                                                          
Brunei       Brunei                              0.979146          1.008723                3.020717
             Cambodia                           17.807929         18.367249                3.140850
             China                            3011.357934       3033.465004                0.734123
             Germany                           523.147319        527.906664                0.909752
             India                             772.462204        437.722493              -43.334122
...                                                   ...               ...                     ...
Vietnam      Singapore                        8971.893722       8971.019269               -0.009747
             Thailand                        16123.525130      16732.175921                3.774924


In [ ]:
agg_trade = ge_model.aggregate_trade_results

print(agg_trade)
# country multilateral resistance (MR) terms
mr_terms = ge_model.country_mr_terms
# Get the solver diaganoistics, which is a dictionary containing many types of solver diagnostic info
solver_diagnostics = ge_model.solver_diagnostics

                      baseline modeled shipments  experiment shipments  shipments change (percent)  baseline modeled foreign exports  experiment foreign exports  foreign exports change (percent)  baseline modeled consumption  experiment consumption  consumption change (percent)  baseline modeled foreign imports  experiment foreign imports  foreign imports change (percent)  baseline modeled intranational trade  experiment modeled intranational trade  intranational trade change (percent)
country                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                   
Brunei            

In [ ]:
mr_terms = ge_model.country_mr_terms
print(mr_terms)

                      baseline imr  conditional imr  experiment imr  baseline omr  conditional omr  experiment omr
country                                                                                                           
Brunei                    1.004164         0.997976        0.997976      0.834301         0.851428        0.851428
Cambodia                  0.991985         0.986263        0.986263      0.824599         0.841880        0.841880
China                     1.097498         1.082541        1.082541      0.911384         0.922943        0.922943
Germany                   1.026177         1.012771        1.012771      0.851518         0.862790        0.862790
India                     1.000000         1.000000        1.000000      0.832179         0.852149        0.852149
Indonesia                 1.027662         1.022257        1.022257      0.853696         0.872038        0.872038
Lao PDR                   0.982778         0.977423        0.977423      0.81753

In [ ]:
solver_diagnostics = ge_model.solver_diagnostics

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
# Import the necessary library
import os

# Specify the directory path within Colab where you want to save the results
output_directory = "/content/drive/My Drive/"

# Make sure the directory exists, or create it
os.makedirs(output_directory, exist_ok=True)

# Export the results using ge_model.export_results
ge_model.export_results(directory=output_directory, name="GE_analysis")